In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-254250
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-254250


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cpu_cluster_name = "udacity-project"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-03-04T13:36:33.128000+00:00', 'errors': None, 'creationTime': '2024-03-04T13:36:23.866110+00:00', 'modifiedTime': '2024-03-04T13:36:34.351061+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'Standard_DS3_v2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', 
                      script='train.py', 
                      arguments =[],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=4,
                             max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config = hyperdrive_config)

# Show run details with the Jupyter widget

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442
Web View: https://ml.azure.com/runs/HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-254250/workspaces/quick-starts-ws-254250&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-03-04T13:41:31.9890295Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442_3' 
[2024-03-04T13:41:31.5226226Z][SCHEDULER][INFO]Scheduling job, id='HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442_1' 
[2024-03-04T13:41:31.9510884Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442_2' 
[2024-03-04T13:41:30.941002][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-03-04T13:41:31.6528376Z][SCHEDULER][INFO]Scheduling job, id='HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442_2' 
[2024-03-04T13:41:31.9741748Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_1f02430f-c

{'runId': 'HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442',
 'target': 'udacity-project',
 'status': 'Completed',
 'startTimeUtc': '2024-03-04T13:41:30.181502Z',
 'endTimeUtc': '2024-03-04T13:56:06.333072Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '8c08c1fb-0a49-4d98-a23a-c45930705cf6',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '44',
  'best_child_run_id': 'HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442_3',
  'score': '0.9171471927162368',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCli

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

best_run

Best Run Id:  HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442_3
Accuracy: 0.9171471927162367


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_1f02430f-ceb0-4a0a-bd81-3c24e4c3c442_3,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=path)

In [7]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
train_df = pd.concat([x, y], axis=1)
train_dir = './training'
train_df_path = os.path.join(train_dir, 'bankmarketing_train.csv')
train_df.to_csv(train_df_path)
data_store = ws.get_default_datastore()
data_store.upload(src_dir=train_dir, target_path=train_dir)
train_ds = TabularDatasetFactory.from_delimited_files(data_store.path(train_df_path))

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading ./training/bankmarketing_train.csv
Uploaded ./training/bankmarketing_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experi,ment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
# automl_config = AutoMLConfig(
#     experiment_timeout_minutes=30,
#     task=,
#     primary_metric=,
#     training_data=,
#     label_column_name=,
#     n_cross_validations=)
automl_config = AutoMLConfig(
    name="AutoML pipeline",
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_ds,
    label_column_name="y",
    n_cross_validations=5
    )

In [9]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
automl_run

Submitting remote run.
No run_configuration provided, running on udacity-project with default configuration
Running on remote compute: udacity-project


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_2f16c12c-9d11-4ca9-9501-5f05ef28fd9d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_2f16c12c-9d11-4ca9-9501-5f05ef28fd9d,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_2f16c12c-9d11-4ca9-9501-5f05ef28fd9d,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, best_model = automl_run.get_output()
print(best_run)
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")
joblib.dump(best_model, 'outputs/automl_model.joblib')

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: udacity-project,
Id: AutoML_2f16c12c-9d11-4ca9-9501-5f05ef28fd9d_31,
Type: azureml.scriptrun,
Status: Completed)
Best run arguments: []
Best run metrics: {'weighted_accuracy': 0.9615038635549963, 'AUC_micro': 0.981033713194913, 'average_precision_score_macro': 0.8293175966816803, 'recall_score_macro': 0.7414130738505553, 'AUC_weighted': 0.9486464274889205, 'f1_score_macro': 0.7688235155517565, 'accuracy': 0.9177238239757208, 'norm_macro_recall': 0.48282614770111076, 'f1_score_micro': 0.9177238239757208, 'recall_score_micro': 0.9177238239757208, 'average_precision_score_weighted': 0.9565338127968163, 'precision_score_micro': 0.9177238239757208, 'log_loss': 0.27933787882882877, 'precision_score_weighted': 0.9107540497211704, 'matthews_correlation': 0.544927573760795, 'AUC_macro': 0.9486464274889203, 'f1_score_weighted': 0.9127919103364681, 'balanced_accuracy': 0.7414130738505553, 'precision_score_macro': 0.8076758683558648, 'recall_score_weighted': 0.9177238239757208, 'av

['outputs/automl_model.joblib']

In [11]:
compute_target.delete()